In [1]:
from data_preprocess.image_matching_dataset import ImageMatchingDataset
import torchvision.transforms as transforms
import torch

# from feature_descriptors.sift_descriptor import get_keypoint_and_descriptor
# from feature_matching.flann_matcher import flann_matcher
# from load_h5py_files import load_sift_output, load_flann_output, load_flann_from_images_name
from load_h5py_files import load_matches_from_h5
from batch_matcher import process_batches
from clustering.cluster_images import build_graph, graph_clustering
from batch_descriptor import batch_feature_descriptor
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import os
import h5py
import numpy as np
import cv2 as cv
from pathlib import Path

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

/home/jovyan/Image_matching_3d/feature_matching/LightGlue/lightglue/lightglue.py:24: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [2]:
from data_preprocess.pseudo_dataset_matching import PseudoMatchingDataset
from torch.utils.data import DataLoader
import torch

DEVICE = torch.device(0 if torch.cuda.is_available() else 'cpu')


pseudo_matching_sift_dataset = PseudoMatchingDataset(pairs_path='data/train_pairs/data_pairs.csv', descriptors_path='evaluation/sift_descriptors_outputs/merged.h5', descriptor_type='sift', device=DEVICE )

train_sift_loader = DataLoader(pseudo_matching_sift_dataset, batch_size=16, shuffle=True, num_workers=10, pin_memory=True)

In [6]:
next(iter(train_loader))

{'descriptor_type': ['sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift',
  'sift'],
 'dataset': ['pt_brandenburg_british_buckingham',
  'imc2023_heritage',
  'amy_gardens',
  'pt_stpeters_stpauls',
  'pt_brandenburg_british_buckingham',
  'imc2023_theather_imc2024_church',
  'pt_sacrecoeur_trevi_tajmahal',
  'imc2023_heritage',
  'amy_gardens',
  'pt_piazzasanmarco_grandplace',
  'pt_piazzasanmarco_grandplace',
  'pt_brandenburg_british_buckingham',
  'imc2023_heritage',
  'pt_piazzasanmarco_grandplace',
  'pt_stpeters_stpauls',
  'pt_stpeters_stpauls'],
 'image1': ['buckingham_palace_82600926_8760297100.png',
  'outliers_img_8890.png',
  'peach_0097.png',
  'st_pauls_cathedral_72125766_3947493463.png',
  'buckingham_palace_42572558_3847795752.png',
  'church_00047.png',
  'trevi_fountain_54470469_2296508341.png',
  'dioscuri_archive_0004.png',
  'peach_0096.png',
  'piazza_san_marco_2126921

In [3]:
import h5py
from pathlib import Path
from feature_matching.lightglue_matcher import match_features_for_plots
import cv2 as cv
import numpy as np
from batch_matcher import process_batches

# Set up output path
output_dir = Path('evaluation/matches_output')
output_dir.mkdir(exist_ok=True, parents=True)

# Initialize output files for both matchers
flann_sift_output = output_dir / 'sift_flann_matches'


# Run flann matcher
print("\nRunning FLANN matching...")
flann_sift_matches = process_batches(
    train_sift_loader,
    matcher='flann',
    output_path=flann_sift_output
)

print("\nMatching complete!")
print(f"FLANN matches saved to: {flann_sift_output}")

# Print some statistics
print("\nFLANN:")
print(f"Total pairs processed: {len(flann_sift_matches)}")
print(f"Average matches per pair: {np.mean([len(m['matches_idx']) for m in flann_sift_matches]):.1f}")


Running FLANN matching...
Processing 11075 batches with flann...
Batch 1/11075
Batch 2/11075
Batch 3/11075
Batch 4/11075
Batch 5/11075
Batch 6/11075
Batch 7/11075
Batch 8/11075
Batch 9/11075
Batch 10/11075
Saving checkpoint after batch 10
Batch 11/11075
Batch 12/11075
Batch 13/11075
Batch 14/11075
Batch 15/11075
Batch 16/11075
Batch 17/11075
Batch 18/11075
Batch 19/11075
Batch 20/11075
Saving checkpoint after batch 20
Batch 21/11075
Batch 22/11075
Batch 23/11075
Batch 24/11075
Batch 25/11075
Batch 26/11075
Batch 27/11075
Batch 28/11075
Batch 29/11075
Batch 30/11075
Saving checkpoint after batch 30
Batch 31/11075
Batch 32/11075
Batch 33/11075
Batch 34/11075
Batch 35/11075
Batch 36/11075
Batch 37/11075
Batch 38/11075
Batch 39/11075
Batch 40/11075
Saving checkpoint after batch 40
Batch 41/11075
Batch 42/11075
Batch 43/11075
Batch 44/11075
Batch 45/11075
Batch 46/11075
Batch 47/11075
Batch 48/11075
Batch 49/11075
Batch 50/11075
Saving checkpoint after batch 50
Batch 51/11075
Batch 52/1107

/home/jovyan/Image_matching_3d/dlrv/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jovyan/Image_matching_3d/dlrv/lib/python3.11/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [4]:
with h5py.File(flann_sift_output / "11070_flann_sift.h5", "r") as f:
    for key in f.keys():
        num_rows = f[key]
        print(f"{key}: {num_rows} rows")

    print(len(f.keys()))

0: <HDF5 group "/0" (9 members)> rows
1: <HDF5 group "/1" (9 members)> rows
10: <HDF5 group "/10" (9 members)> rows
100: <HDF5 group "/100" (9 members)> rows
101: <HDF5 group "/101" (9 members)> rows
102: <HDF5 group "/102" (9 members)> rows
103: <HDF5 group "/103" (9 members)> rows
104: <HDF5 group "/104" (9 members)> rows
105: <HDF5 group "/105" (9 members)> rows
106: <HDF5 group "/106" (9 members)> rows
107: <HDF5 group "/107" (9 members)> rows
108: <HDF5 group "/108" (9 members)> rows
109: <HDF5 group "/109" (9 members)> rows
11: <HDF5 group "/11" (9 members)> rows
110: <HDF5 group "/110" (9 members)> rows
111: <HDF5 group "/111" (9 members)> rows
112: <HDF5 group "/112" (9 members)> rows
113: <HDF5 group "/113" (9 members)> rows
114: <HDF5 group "/114" (9 members)> rows
115: <HDF5 group "/115" (9 members)> rows
116: <HDF5 group "/116" (9 members)> rows
117: <HDF5 group "/117" (9 members)> rows
118: <HDF5 group "/118" (9 members)> rows
119: <HDF5 group "/119" (9 members)> rows
12: 

In [5]:
with h5py.File(flann_sift_output / "11075_flann_sift.h5", "r") as f:
    for key in f.keys():
        num_rows = f[key]
        print(f"{key}: {num_rows} rows")

    print(len(f.keys()))

0: <HDF5 group "/0" (9 members)> rows
1: <HDF5 group "/1" (9 members)> rows
10: <HDF5 group "/10" (9 members)> rows
11: <HDF5 group "/11" (9 members)> rows
12: <HDF5 group "/12" (9 members)> rows
13: <HDF5 group "/13" (9 members)> rows
14: <HDF5 group "/14" (9 members)> rows
15: <HDF5 group "/15" (9 members)> rows
16: <HDF5 group "/16" (9 members)> rows
17: <HDF5 group "/17" (9 members)> rows
18: <HDF5 group "/18" (9 members)> rows
19: <HDF5 group "/19" (9 members)> rows
2: <HDF5 group "/2" (9 members)> rows
20: <HDF5 group "/20" (9 members)> rows
21: <HDF5 group "/21" (9 members)> rows
22: <HDF5 group "/22" (9 members)> rows
23: <HDF5 group "/23" (9 members)> rows
24: <HDF5 group "/24" (9 members)> rows
25: <HDF5 group "/25" (9 members)> rows
26: <HDF5 group "/26" (9 members)> rows
27: <HDF5 group "/27" (9 members)> rows
28: <HDF5 group "/28" (9 members)> rows
29: <HDF5 group "/29" (9 members)> rows
3: <HDF5 group "/3" (9 members)> rows
30: <HDF5 group "/30" (9 members)> rows
31: <HDF

In [6]:
from load_h5py_files import load_matches_from_h5

load_matches_from_h5(flann_sift_output / "2040_flann_sift.h5")

[{'image1': 'outliers_dsc_4115_thumb.png',
  'image2': 'dioscuri_img_0246.png',
  'dataset': 'imc2023_heritage',
  'points0': tensor([[820.2991, 304.5810]]),
  'points1': tensor([[661.9404, 696.1146]]),
  'features1': {'scene': 'outliers'},
  'features2': {'scene': 'dioscuri'},
  'matches_idx': tensor([[1560, 1109]]),
  'distances': tensor([0.3669])},
 {'image1': 'st_pauls_cathedral_01207461_5308976270.png',
  'image2': 'st_peters_square_16459160_169609859.png',
  'dataset': 'pt_stpeters_stpauls',
  'points0': tensor([[718.8984, 287.4998],
          [839.5784, 230.3611],
          [725.1885, 338.5907],
          [651.7815, 414.0165],
          [206.5593, 299.6302],
          [ 39.2943, 597.4852],
          [730.1702, 383.3237],
          [771.5795, 139.7433],
          [612.0940, 412.6653],
          [384.9335, 521.6953],
          [868.1903, 240.1417],
          [284.1153, 243.7653],
          [453.6143, 406.4623],
          [290.1106, 515.9416],
          [729.1281, 386.3857],
      

## with disk

In [7]:
from data_preprocess.pseudo_dataset_matching import PseudoMatchingDataset
from torch.utils.data import DataLoader
import torch

DEVICE = torch.device(0 if torch.cuda.is_available() else 'cpu')


pseudo_matching_disk_dataset = PseudoMatchingDataset(pairs_path='data/train_pairs/data_pairs.csv', descriptors_path='evaluation/disk_descriptors_outputs/merged.h5', descriptor_type='disk', device=DEVICE)

train_disk_loader = DataLoader(pseudo_matching_disk_dataset, batch_size=16, shuffle=True, num_workers=10, pin_memory=True)

In [5]:
next(iter(train_loader))

{'descriptor_type': ['disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk',
  'disk'],
 'dataset': ['imc2023_heritage',
  'pt_sacrecoeur_trevi_tajmahal',
  'pt_brandenburg_british_buckingham',
  'imc2024_lizard_pond',
  'imc2024_lizard_pond',
  'imc2024_dioscuri_baalshamin',
  'imc2023_heritage',
  'pt_brandenburg_british_buckingham',
  'imc2024_lizard_pond',
  'amy_gardens',
  'pt_brandenburg_british_buckingham',
  'pt_stpeters_stpauls',
  'pt_brandenburg_british_buckingham',
  'imc2023_heritage',
  'pt_stpeters_stpauls',
  'imc2023_heritage'],
 'image1': ['outliers_img_8662.png',
  'trevi_fountain_59883665_3581996971.png',
  'buckingham_palace_82600926_8760297100.png',
  'outliers_img_20230617_181852.png',
  'pond_00440.png',
  'baalshamin_19739156806_e4f69d8792_o.png',
  'dioscuri_img_0348.png',
  'british_museum_67471600_1224757581.png',
  'lizard_00618.png',
  'peach_0116.png',
  'british_

In [8]:
import h5py
from pathlib import Path
from feature_matching.lightglue_matcher import match_features_for_plots
import cv2 as cv
import numpy as np
from batch_matcher import process_batches

# Set up output path
output_dir = Path('evaluation/matches_output')
output_dir.mkdir(exist_ok=True, parents=True)

# Initialize output files for both matchers
flann_disk_output = output_dir / 'disk_flann_matches'


# Run flann matcher
print("\nRunning FLANN matching...")
flann_disk_matches = process_batches(
    train_disk_loader,
    matcher='flann',
    output_path=flann_disk_output
)

print("\nMatching complete!")
print(f"FLANN matches saved to: {flann_disk_output}")

# Print some statistics
print("\nFLANN:")
print(f"Total pairs processed: {len(flann_disk_matches)}")
print(f"Average matches per pair: {np.mean([len(m['matches_idx']) for m in flann_disk_matches]):.1f}")


Running FLANN matching...
Processing 11075 batches with flann...
Batch 1/11075
Batch 2/11075
Batch 3/11075
Batch 4/11075
Batch 5/11075
Batch 6/11075
Batch 7/11075
Batch 8/11075
Batch 9/11075
Batch 10/11075
Saving checkpoint after batch 10
Batch 11/11075
Batch 12/11075
Batch 13/11075
Batch 14/11075
Batch 15/11075
Batch 16/11075
Batch 17/11075
Batch 18/11075
Batch 19/11075
Batch 20/11075
Saving checkpoint after batch 20
Batch 21/11075
Batch 22/11075
Batch 23/11075
Batch 24/11075
Batch 25/11075
Batch 26/11075
Batch 27/11075
Batch 28/11075
Batch 29/11075
Batch 30/11075
Saving checkpoint after batch 30
Batch 31/11075
Batch 32/11075
Batch 33/11075
Batch 34/11075
Batch 35/11075
Batch 36/11075
Batch 37/11075
Batch 38/11075
Batch 39/11075
Batch 40/11075
Saving checkpoint after batch 40
Batch 41/11075
Batch 42/11075
Batch 43/11075
Batch 44/11075
Batch 45/11075
Batch 46/11075
Batch 47/11075
Batch 48/11075
Batch 49/11075
Batch 50/11075
Saving checkpoint after batch 50
Batch 51/11075
Batch 52/1107

/home/jovyan/Image_matching_3d/dlrv/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/jovyan/Image_matching_3d/dlrv/lib/python3.11/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [9]:
from load_h5py_files import load_matches_from_h5

load_matches_from_h5(flann_disk_output / "10_flann_disk.h5")

[{'image1': 'vineyard_split_2_frame_1165.png',
  'image2': 'vineyard_split_3_frame_0200.png',
  'dataset': 'fbk_vineyard',
  'points0': tensor([], size=(0, 2)),
  'points1': tensor([], size=(0, 2)),
  'features1': {'scene': 'vineyard_split_2'},
  'features2': {'scene': 'vineyard_split_3'},
  'matches_idx': tensor([], size=(0, 2), dtype=torch.int64),
  'distances': tensor([])},
 {'image1': 'trevi_fountain_01241225_4293371290.png',
  'image2': 'sacre_coeur_41224690_4580747259.png',
  'dataset': 'pt_sacrecoeur_trevi_tajmahal',
  'points0': tensor([], size=(0, 2)),
  'points1': tensor([], size=(0, 2)),
  'features1': {'scene': 'trevi_fountain'},
  'features2': {'scene': 'sacre_coeur'},
  'matches_idx': tensor([], size=(0, 2), dtype=torch.int64),
  'distances': tensor([])},
 {'image1': 'piazza_san_marco_49865216_1748767363.png',
  'image2': 'grand_place_brussels_36406299_8032284977.png',
  'dataset': 'pt_piazzasanmarco_grandplace',
  'points0': tensor([], size=(0, 2)),
  'points1': tensor([